# Практика курсової "Порівняння градієнтних методів оптимізації функцій кількох змінних"

## Вибір функції

Оберемо одну з класичних функцій для тестування оптимізаційних методів:    https://en.wikipedia.org/wiki/Test_functions_for_optimization

Працюватимемо з функцією Хіммельблау, що має 4 глобальні мінімума:    https://en.wikipedia.org/wiki/Himmelblau%27s_function
але ми можемо обрати будь-яку і запустити на ній наш алгоритм.

## Знаходження часткових похідних за допомогою символьних обчисленнь Sympy

In [1]:
import sympy as sp

print('Обрана функція: (x**2 + y - 11)**2 + (x + y**2 - 7)**2', '\n')
x, y = sp.symbols('x y')
print('Часткова похідна функції по х:', sp.diff((x**2 + y - 11)**2 + (x + y**2 - 7)**2, x))
print('Часткова похідна функції по у:', sp.diff((x**2 + y - 11)**2 + (x + y**2 - 7)**2, y))

Обрана функція: (x**2 + y - 11)**2 + (x + y**2 - 7)**2 

Часткова похідна функції по х: 4*x*(x**2 + y - 11) + 2*x + 2*y**2 - 14
Часткова похідна функції по у: 2*x**2 + 4*y*(x + y**2 - 7) + 2*y - 22


## Градієнтні спуски

In [5]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np


def f(x:float, y:float):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2


def df_dx(x:float, y:float):
    return 4*x*(x**2 + y - 11) + 2*x + 2*y**2 - 14


def df_dy(x:float, y:float):
    return 2*x**2 + 4*y*(x + y**2 - 7) + 2*y - 22


# plot_3d_surface() start
Contour=False
x_plt = np.arange(-4, 4, 0.05)
y_plt = np.arange(-4, 4, 0.05)
Z_plt = np.array([[f(x, y) for x in x_plt] for y in y_plt])

trace1 = go.Contour(x=x_plt, y=y_plt, z=Z_plt)

#   створення даних для побудування фігури;   оптимальна палітра для конкретного графіку
trace2 = go.Surface(x=x_plt, y=y_plt, z=Z_plt, colorbar_x=-0.5)

data1 = [trace1]
data2 = [trace2]

fig1 = go.Figure(data = data1)
fig2 = go.Figure(data = data2)

# додання ліній рівня на об'ємний графік
fig2.update_traces(contours_z=dict(show=True, usecolormap=True,
                              highlightcolor="limegreen", project_z=True))

# plot_3d_surface() end


def drawing_scatter3d(x_y_array, color='red', name=''):
    """Функція відмальовки кроків град.спуску на 3-вимірному графіку за заданною відповіддю алгоритма"""
    fig2.add_trace(
    go.Scatter3d(x=x_y_array[:,0],
                 y=x_y_array[:,1], 
                 z=f(x_y_array[:,0], x_y_array[:,1]),
                 name=name,
                 line=dict(width=2, color=color)))
    
    fig2.update_layout(
        title=dict(text='Function'),
        title_font_size=40,
        margin=dict(l=0, r=0, t=60, b=0), 
        height=650)
    
    
def drawing_scatter2d(x_y_array, color='red', name=''):
    """Функція відмальовки кроків град.спуску на графіку ліній рівня функції за заданною відповіддю алгоритма"""
    fig1.add_trace(
    go.Scatter(x=x_y_array[:,0],
               y=x_y_array[:,1], 
               name=name,
               line=dict(width=2, color=color)))
    
    fig1.update_layout(
        title=dict(text='Level lines'),
        title_font_size=40,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01 ),
        height=800)
    

# методи оптимізації      
def classic_grad_descent(x, y, lmd_x, lmd_y, number_of_iterations):
    """Классичний градієнтний спуск з константними заданними кроками та наперед заданною кількістю ітерацій"""
    dots = np.array([x, y], dtype=float).reshape(-1, 2)

    for n in range(number_of_iterations):
        x = x - lmd_x * df_dx(x, y)
        y = y - lmd_y * df_dy(x, y)

        dots = np.append(dots, [[x, y]], axis=0)
        
    return dots


def nesterov_grad_descent(x, y, alpha, beta, number_of_iterations):
    """Accelerated gradient descent"""
    dots = np.array([x, y], dtype=float).reshape(-1, 2)
    
    x = x - alpha * df_dx(x, y)
    y = y - alpha * df_dy(x, y)
    dots = np.append(dots, [[x, y]], axis=0)
    
    for k in range(2, number_of_iterations):
        x = x - alpha * df_dx(x + beta*(x - dots[k-1, 0]), y) + beta*(x-dots[k-1, 0])
        y = y - alpha * df_dy(x, y + beta*(y - dots[k-1, 1]))  + beta*(y-dots[k-1, 1])

        dots = np.append(dots, [[x, y]], axis=0)
        
    return dots


def stohastic_grad_descent(x, y, lmd, number_of_iterations):
    """Стохастичний градієнтний спуск з константними заданними кроками та наперед заданною кількістю ітерацій"""
    dots = np.array([x, y], dtype=float).reshape(-1, 2)
    
    for n in range(number_of_iterations):
        if np.random.randint(0,2) == 0:
            x = x - lmd * df_dx(x, y)
        else:
            y = y - lmd * df_dy(x, y)

        dots = np.append(dots, [[x, y]], axis=0)
        
    return dots

In [6]:
drawing_scatter3d(
    x_y_array=classic_grad_descent(x=0.2, y=-3.65, lmd_x=0.01, lmd_y = 0.01, number_of_iterations=9),
    name='classic_grad_descent',
    color='green')

drawing_scatter3d(
    x_y_array=nesterov_grad_descent(x=0.2, y=-3.65, alpha=0.02, beta = 0.03, number_of_iterations=6),
    name='nesterov_grad_descent',
    color='lightgreen')

drawing_scatter3d(
    x_y_array=stohastic_grad_descent(x=0.2, y=-3.65, lmd=0.01, number_of_iterations=20),
    name='stohastic_grad_descent',
    color='yellowgreen')

drawing_scatter2d(
    x_y_array=classic_grad_descent(x=0.2, y=-3.65, lmd_x=0.01, lmd_y = 0.01, number_of_iterations=9),
    name='classic_grad_descent',
    color='green')

drawing_scatter2d(
    x_y_array=nesterov_grad_descent(x=0.2, y=-3.65, alpha=0.02, beta = 0.03, number_of_iterations=6),
    name='nesterov_grad_descent',
    color='lightgreen')

drawing_scatter2d(
    x_y_array=stohastic_grad_descent(x=0.2, y=-3.65, lmd=0.01, number_of_iterations=20),
    name='stohastic_grad_descent',
    color='yellowgreen')

fig1.show()
fig2.show()

Бачимо, що результат роботи кожного з алгоритмів залежить від обраного початкового наближення та гіперпараметрів. 

1) Стохастичному градієнтному спуску потрібно більше ітерацій, проте він має вдвічі меньше обчислень, ніж классичному
2) Метод Нестерова завдяки коєфіцієнтам $\alpha$ та $\beta$ і використанням попередніх результатів значно меньше відхиляється від своєї траєкторії та швидше сходиться до мінімуму.

In [8]:
$jupyter nbconvert --to html notebook.ipynb

SyntaxError: invalid syntax (<ipython-input-8-2fc3be4f2c33>, line 1)